<a href="https://colab.research.google.com/github/adrincont/Tareas_UNAL/blob/main/Tareas/Tarea1_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recorrido por Colab (Punto 2 GPU)


1.   Transcriba el siguiente código en Colab son sus propias manos.

In [1]:
# Liberias a usar
%tensorflow_version 2.x
import tensorflow as tf
print('Versión de Tensorflow: ' + tf.__version__)
import numpy as np
import timeit
import os
import gzip
from google.colab import drive
drive.mount('/content/gdrive')
ruta = '/content/gdrive/MyDrive/Colab Notebooks'

Versión de Tensorflow: 2.8.0
Mounted at /content/gdrive


## Lectura del set de datos

In [2]:
# Fucnion para la lectura de los datos de entrenamiento y validacion
def load_set(ruta, tipo='train'):
  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)
  with gzip.open(ruta_categorias) as rut_cat:
      etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)
  with gzip.open(ruta_imagenes) as rut_imgs:
      imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)
  return imagenes, etiquetas
# Leer los archivos del Drive
ruta_datos = ruta + '/datos'
# Conjuntos de entrenamiento y test para el modelo
X_train, Y_train = load_set(ruta_datos, tipo='train')
X_test, Y_test = load_set(ruta_datos, tipo='t10k')
# Configurar los tamañso de los datos
X_train = X_train[0:59904:]
X_test = X_test[0:9984:]
Y_train = Y_train[0:59904:]
Y_test = Y_test[0:9984]
X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

## Modelo de la red

In [3]:
tf.random.set_seed(1997)
model = tf.keras.models.Sequential()
#-------------------------------------------------------------------------------
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))
#-------------------------------------------------------------------------------
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
#-------------------------------------------------------------------------------
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))
#-------------------------------------------------------------------------------
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))
#-------------------------------------------------------------------------------
model.summary()
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 28, 28, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                        

## Entrenamiento con GPU

In [4]:
import timeit
# Guardar puntos de control durante el entrenamiento
checkpoint_path = ruta + "/Modelo_Punto2_GPU.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Crear una devolución de llamada que guarde los pesos del modelo
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)
# Entrenar al modelo
def entrenamiento_gpu():
  with tf.device('/device:GPU:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1,callbacks=[cp_callback])
  return None
cpu_time = timeit.timeit('entrenamiento_gpu()', number=1, setup='from __main__ import entrenamiento_gpu')

Epoch 1/2
467/468 [============================>.] - ETA: 0s - loss: 0.7000 - accuracy: 0.7764
Epoch 1: saving model to /content/gdrive/MyDrive/Colab Notebooks/Modelo_Punto2_GPU.ckpt
468/468 [==============================] - 31s 41ms/step - loss: 0.6993 - accuracy: 0.7766 - val_loss: 0.3565 - val_accuracy: 0.8728
Epoch 2/2
468/468 [==============================] - ETA: 0s - loss: 0.3927 - accuracy: 0.8623
Epoch 2: saving model to /content/gdrive/MyDrive/Colab Notebooks/Modelo_Punto2_GPU.ckpt
468/468 [==============================] - 19s 40ms/step - loss: 0.3927 - accuracy: 0.8623 - val_loss: 0.3187 - val_accuracy: 0.8870


In [ ]:
# Cargar pesos de un modelo guardado (Esta linea se corre solo para retornar modelo a un punto anterior)
model.load_weights(checkpoint_path)
cpu_time = timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu')